In [94]:
import os

In [95]:
%pwd

'c:\\'

In [96]:
os.chdir("../")

In [97]:
%pwd

'c:\\'

In [98]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [99]:
from GGDDClassifier.constants import *
from GGDDClassifier.utils.common import read_yaml, create_directories

In [100]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      


In [101]:
import os
import urllib.request as request
import zipfile
from GGDDClassifier import logger
from GGDDClassifier.utils.common import get_size

In [102]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [119]:
import os
import yaml

try:
    with open('config/config.yaml', 'r') as f:
        data_ingestion_config = yaml.safe_load(f)
    
    # Specify the directory path
    log_directory = 'research/logs'
    
    # Create the directory if it doesn't exist
    os.makedirs(log_directory, exist_ok=True)
    
    # Add 'local_data_file' attribute to the configuration dictionary
    data_ingestion_config['local_data_file'] = os.path.join(log_directory, 'data_file.txt')
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print("An error occurred:", e)
    raise e


An error occurred: 'dict' object has no attribute 'local_data_file'


AttributeError: 'dict' object has no attribute 'local_data_file'